Probabilistic Sharpe Ratio: Marcus Lopez de Prado. 

Kurtosis: Measure of the "tailedness" of the probability distribution of a real-valued random variable.

Positive Kurtosis: Peak is narrower and higher than normal dist
Negative Kurtosis: Peak is flater and lower than normal dist

Skew: measure of asymetry of the probability distribution of a real-valued random variable
symetrical dist, positive skew (skewed right), negative skew(skewed left)
google image search this

Probabilistc sharpe ratio uses skew and kurtosis to return a Sharpe Ratio in probability terms (not just historical)




In [15]:
import pandas_datareader as pdr
import datetime as dt
import scipy.stats
import numpy as np


In [16]:
ticker = "MSFT"
start = dt.datetime(2021, 2, 26)
end = dt.datetime(2022, 3, 1)
msft = pdr.get_data_yahoo(ticker, start, end)

In [17]:
msft['Daily Return'] = msft['Adj Close'].pct_change(1)
#drop the first row
msft.dropna(inplace=True)

In [18]:
msft

High         Low        Open       Close      Volume  \
Date                                                                     
2021-03-01  237.470001  233.149994  235.899994  236.940002  25324000.0   
2021-03-02  237.300003  233.449997  237.009995  233.869995  22812500.0   
2021-03-03  233.580002  227.259995  232.160004  227.559998  34029500.0   
2021-03-04  232.490005  224.259995  226.740005  226.729996  44727800.0   
2021-03-05  233.270004  226.460007  229.520004  231.600006  41872800.0   
...                ...         ...         ...         ...         ...   
2022-02-23  291.700012  280.100006  290.179993  280.269989  37811200.0   
2022-02-24  295.160004  271.519989  272.510010  294.589996  56989700.0   
2022-02-25  297.630005  291.649994  295.140015  297.309998  32546700.0   
2022-02-28  299.140015  293.000000  294.309998  298.790009  34585700.0   
2022-03-01  299.970001  292.149994  296.399994  294.950012  31217800.0   

             Adj Close  Daily Return  
Date                                  
2021-03-01  235.025620      0.019623  
2021-03-02  231.980423     -0.012957  
2021-03-03  225.721405     -0.026981  
2021-03-04  224.898102     -0.003647  
2021-03-05  229.728760      0.021479  
...                ...           ...  
2022-02-23  280.269989     -0.025893  
2022-02-24  294.589996      0.051094  
2022-02-25  297.309998      0.009233  
2022-02-28  298.790009      0.004978  
2022-03-01  294.950012     -0.012852  

[254 rows x 7 columns]

In [19]:
msft_return_std_dev = msft['Daily Return'].std()
msft_return_std_dev

0.01434722038539185

In [22]:
def compute_sharpe_ratio_annual(df, risk_free_rate=0):
    """ (daily return mean - risk free rate) / by daily return std dev"""
    std = df['Daily Return'].std()
    mean = df['Daily Return'].mean()
    sharpe_ratio = (mean - risk_free_rate) / std
    #annualize it by 252 trading days
    annual_sharpe_ratio = sharpe_ratio * np.sqrt(252)
    return annual_sharpe_ratio

In [25]:
sharpe_ratio_msft_ann = compute_sharpe_ratio_annual(msft)
sharpe_ratio_msft_ann

1.1879948423698579

In [26]:
#for our calculation it is simpler to do daily----jsut put it there for completness.
def compute_sharpe_ratio(df, risk_free_rate=0):
    """ (daily return mean - risk free rate) / by daily return std dev"""
    std = df['Daily Return'].std()
    mean = df['Daily Return'].mean()
    sharpe_ratio = (mean - risk_free_rate) / std
    return sharpe_ratio

In [27]:
sharpe_ratio_msft = compute_sharpe_ratio(msft)
sharpe_ratio_msft

0.07483664075566722

In [30]:
def compute_prob_sharpe_ratio(df, benchmark=0):
    sr = compute_sharpe_ratio(df)
    skew = scipy.stats.skew(df['Daily Return'])
    kurtosis = scipy.stats.kurtosis(df['Daily Return'], fisher=True)
    n = len(df)
    
    std = ( (1 / (n-1)) * (1 + 0.5 * sr**2 - skew * sr + (kurtosis / 4) * sr**2))**0.5
    
    ratio = (sr - benchmark) / std
    prob_sharpe_ratio = scipy.stats.norm.cdf(ratio)
    return prob_sharpe_ratio






In [33]:
sr_microsoft = compute_prob_sharpe_ratio(msft)

print(f"The daily probabilistic sharpe ratio for Microsoft is: {sr_microsoft}")
annual = sr_microsoft*(252**0.5)
print(f"The annualized probabilistic sharpe ratio for Microsoft is: {annual}")

The daily probabilistic sharpe ratio for Microsoft is: 0.8813601262633288
The annualized probabilistic sharpe ratio for Microsoft is: 13.991158257487532
